In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt

In [55]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

In [3]:
prices_day_1_df = pd.read_csv('../data/round-2-island-data-bottle/prices_round_2_day_0.csv', index_col='timestamp', sep=';')
prices_day_neg1_df = pd.read_csv('../data/round-2-island-data-bottle/prices_round_2_day_-1.csv', index_col='timestamp', sep=';')
prices_day_0_df = pd.read_csv('../data/round-2-island-data-bottle/prices_round_2_day_1.csv', index_col='timestamp', sep=';')

In [10]:
Data_df = pd.concat([prices_day_neg1_df,prices_day_0_df,prices_day_1_df],axis = 0)

In [11]:
Data_df.head()

,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY
timestamp,,,,,,,
0,1200.00,1.5,10.5,-2.0,2500.0000,79.00000,-1
100,1201.75,1.5,9.5,-2.0,2499.4197,79.00410,-1
200,1201.75,1.5,9.5,-2.0,2498.8457,79.00821,-1
300,1201.75,1.5,9.5,-2.0,2498.2780,79.01234,-1
400,1201.75,1.5,9.5,-2.0,2497.7166,79.01649,-1


In [13]:
prices_3days_arr = Data_df.ORCHIDS.values

In [19]:
returns_3days_arr = Data_df.ORCHIDS.pct_change().dropna()

In [32]:
# passing the extracted passengers count to adfuller function.
# result of adfuller function is stored in a res variable
res = adfuller(prices_3days_arr)
 
# Printing the statistical result of the adfuller test
print('Augmneted Dickey_fuller Statistic - ORCHIDS: %f' % res[0])
print('p-value: %f' % res[1])
 
# printing the critical values at different alpha levels.
print('critical values at different levels:')
for k, v in res[4].items():
    print('\t%s: %.3f' % (k, v))

Augmneted Dickey_fuller Statistic - ORCHIDS: -1.301728
p-value: 0.628381
critical values at different levels:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [33]:
# passing the extracted passengers count to adfuller function.
# result of adfuller function is stored in a res variable
res = adfuller(returns_3days_arr)
 
# Printing the statistical result of the adfuller test
print('Augmneted Dickey_fuller Statistic - ORCHIDS simple return: %f' % res[0])
print('p-value: %f' % res[1])
 
# printing the critical values at different alpha levels.
print('critical values at different levels:')
for k, v in res[4].items():
    print('\t%s: %.3f' % (k, v))

Augmneted Dickey_fuller Statistic - ORCHIDS simple return: -123.140894
p-value: 0.000000
critical values at different levels:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [30]:
# passing the extracted passengers count to adfuller function.
# result of adfuller function is stored in a res variable
res = adfuller(Data_df.SUNLIGHT.values)
 
# Printing the statistical result of the adfuller test
print('Augmneted Dickey_fuller Statistic - SUNLIGHT: %f' % res[0])
print('p-value: %f' % res[1])
 
# printing the critical values at different alpha levels.
print('critical values at different levels:')
for k, v in res[4].items():
    print('\t%s: %.3f' % (k, v))

Augmneted Dickey_fuller Statistic - SUNLIGHT: -1.350860
p-value: 0.605595
critical values at different levels:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [31]:
# passing the extracted passengers count to adfuller function.
# result of adfuller function is stored in a res variable
res = adfuller(Data_df.HUMIDITY.values)
 
# Printing the statistical result of the adfuller test
print('Augmneted Dickey_fuller Statistic - HUMIDITY: %f' % res[0])
print('p-value: %f' % res[1])
 
# printing the critical values at different alpha levels.
print('critical values at different levels:')
for k, v in res[4].items():
    print('\t%s: %.3f' % (k, v))

Augmneted Dickey_fuller Statistic - HUMIDITY: -1.131613
p-value: 0.702302
critical values at different levels:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [23]:
ts.coint(Data_df.SUNLIGHT, Data_df.ORCHIDS)

(-2.1507203239055794,
 0.4499021276879138,
 array([-3.89680508, -3.33633366, -3.04459137]))

In [24]:
ts.coint(Data_df.HUMIDITY, Data_df.ORCHIDS)

(-2.0810023026459232,
 0.4863006246382938,
 array([-3.89680508, -3.33633366, -3.04459137]))

In [25]:
ts.coint(Data_df.HUMIDITY, Data_df.SUNLIGHT)

(-1.2954306571133984,
 0.8311101858829645,
 array([-3.89680508, -3.33633366, -3.04459137]))

In [26]:
ts.coint(Data_df.SUNLIGHT, Data_df.HUMIDITY)

(-1.3799056718498104,
 0.8043179814856429,
 array([-3.89680508, -3.33633366, -3.04459137]))

In [39]:
def custom_accuracy(y_true, y_pred, tolerance=0.02):
    """
    Calculate the percentage of predictions within a certain tolerance of the actual values.
    :param y_true: The actual values.
    :param y_pred: The predicted values.
    :param tolerance: The percentage tolerance for considering a prediction 'accurate'.
    :return: The accuracy rate as a percentage.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    within_tolerance = np.abs(y_true - y_pred) <= tolerance * np.abs(y_true)
    accuracy = np.mean(within_tolerance)
    return accuracy * 100

### PLS - Partial Least Square

In [44]:
X = Data_df[['SUNLIGHT', 'HUMIDITY']].pct_change().dropna() * 100
y = Data_df['ORCHIDS'].pct_change().dropna() * 100

In [54]:
scipy.stats.normaltest(y).pvalue

0.0

In [49]:
sunlight_normaltest_res = scipy.stats.normaltest(X['SUNLIGHT'])

In [50]:
sunlight_normaltest_res.pvalue

0.0

In [52]:
scipy.stats.normaltest(X['HUMIDITY']).pvalue

0.0

In [45]:
tscv = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    pls = PLSRegression(n_components=2)
    pls.fit(X_train, y_train)
    y_pred = pls.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE: %f" % (rmse))
    mean_ae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error:", mean_ae)

    accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03)
    print("Accuracy",accuracy_rate)


RMSE: 0.126104
Mean Absolute Error: 0.06830370063405794
Accuracy 0.0
RMSE: 0.206803
Mean Absolute Error: 0.08363872109898254
Accuracy 0.0
RMSE: 0.101411
Mean Absolute Error: 0.07911591732738389
Accuracy 0.4


In [57]:
tscv = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train) 
    X_test = scaler.transform(X_test) 

    pls = PLSRegression(n_components=2)
    pls.fit(X_train, y_train)
    y_pred = pls.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE: %f" % (rmse))
    mean_ae = mean_absolute_error(y_test, y_pred)
    print("Mean Absolute Error:", mean_ae)

    accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03)
    print("Accuracy",accuracy_rate)


RMSE: 0.126104
Mean Absolute Error: 0.06830370063405794
Accuracy 0.0
RMSE: 0.206803
Mean Absolute Error: 0.08363872109898254
Accuracy 0.0
RMSE: 0.101411
Mean Absolute Error: 0.07911591732738389
Accuracy 0.4


In [77]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20001, 3), (10001, 3), (20001,), (10001,))

In [194]:
train_test_split_idx = int(X.shape[0]*2/3)
X_train, X_test = X.iloc[:train_test_split_idx+1], X.iloc[train_test_split_idx+1:]
y_train, y_test = y.iloc[:train_test_split_idx+1], y.iloc[train_test_split_idx+1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


ols = sm.OLS(y_train, X_train)
#ols.fit(X_train, y_train)
results = ols.fit()

y_pred = results.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
mean_ae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_ae)

accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03/100)
print("Accuracy",accuracy_rate)


RMSE: 0.099549
Mean Absolute Error: 0.07747564394208942
Accuracy 0.0


In [195]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ORCHIDS   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     7069.
Date:                Sat, 13 Apr 2024   Prob (F-statistic):               0.00
Time:                        23:52:46   Log-Likelihood:                 17723.
No. Observations:               20002   AIC:                        -3.544e+04
Df Residuals:                   19999   BIC:                        -3.542e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.001     -0.419      0.675      -0.002       0.001
x1             0.5478      0.012     44.308      0.000       0.524       0.572
x2            -0.4690      0.012    -37.940      0.000      -0.493      -0.445
==============================================================================
Omnibus:                    14797.416   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5753368.361
Skew:                          -2.427   Prob(JB):                         0.00
Kurtosis:                      85.944   Cond. No.                         35.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
def transform_humidity(val):
    if val > 80:
        return 80-val
    elif 0 <= val < 80:
        return val-60
    else:
        return 0

In [86]:
Data_df['HUMIDITY_60_80_trans'] = Data_df['HUMIDITY'].apply(transform_humidity)

In [93]:
Data_df.HUMIDITY.describe()

count    30003.000000
mean        79.073562
std          9.366057
min         59.999580
25%         71.204712
50%         78.579020
75%         86.771817
max         97.513270
Name: HUMIDITY, dtype: float64

In [94]:
Data_df.HUMIDITY_60_80_trans.describe()

count    30003.000000
mean         2.643358
std         10.767233
min        -17.513270
25%         -6.771817
50%          4.788540
75%         12.084263
max         19.999800
Name: HUMIDITY_60_80_trans, dtype: float64

In [100]:
X_2.HUMIDITY_60_80_trans.describe()

/opt/miniconda3/envs/imc_env/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


count    3.000300e+04
mean              NaN
std               NaN
min              -inf
25%     -7.065055e-02
50%     -1.859567e-02
75%      5.101466e-02
max               inf
Name: HUMIDITY_60_80_trans, dtype: float64

In [103]:
X_2 = Data_df[['SUNLIGHT', 'HUMIDITY_60_80_trans']].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0) * 100
y_2 = Data_df['ORCHIDS'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0) * 100

In [104]:
X_2.HUMIDITY_60_80_trans.describe()

count     30003.000000
mean        -26.897811
std        3439.069785
min     -526119.473684
25%          -0.070639
50%          -0.018591
75%           0.051007
max         125.524476
Name: HUMIDITY_60_80_trans, dtype: float64

In [196]:
train_test_split_idx = int(X.shape[0]*2/3)
X_train, X_test = X_2.iloc[:train_test_split_idx+1], X_2.iloc[train_test_split_idx+1:]
y_train, y_test = y_2.iloc[:train_test_split_idx+1], y_2.iloc[train_test_split_idx+1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


ols = sm.OLS(y_train, X_train)
#ols.fit(X_train, y_train)
results2 = ols.fit()

y_pred = results2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
mean_ae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_ae)

accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03/100)
print("Accuracy",accuracy_rate)


RMSE: 0.172949
Mean Absolute Error: 0.07487941995816653
Accuracy 0.0


In [197]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ORCHIDS   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     6373.
Date:                Sat, 13 Apr 2024   Prob (F-statistic):               0.00
Time:                        23:52:57   Log-Likelihood:                 19059.
No. Observations:               20002   AIC:                        -3.811e+04
Df Residuals:                   19999   BIC:                        -3.809e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.001     -1.008      0.314      -0.002       0.001
x1             0.0745      0.001    112.896      0.000       0.073       0.076
x2         -6.981e-05      0.001     -0.106      0.916      -0.001       0.001
==============================================================================
Omnibus:                      196.805   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              329.591
Skew:                           0.024   Prob(JB):                     2.69e-72
Kurtosis:                       3.627   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [112]:
X_3 = Data_df[['SUNLIGHT']].pct_change().dropna() * 100
y_3 = Data_df['ORCHIDS'].pct_change().dropna() * 100

In [190]:
train_test_split_idx = int(X.shape[0]*2/3)
X_train, X_test = X_3.iloc[:train_test_split_idx+1], X_3.iloc[train_test_split_idx+1:]
y_train, y_test = y_3.iloc[:train_test_split_idx+1], y_3.iloc[train_test_split_idx+1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


ols = sm.OLS(y_train, X_train)
#ols.fit(X_train, y_train)
results3 = ols.fit()

y_pred = results3.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
mean_ae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_ae)

accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03)
print("Accuracy",accuracy_rate)


RMSE: 0.096111
Mean Absolute Error: 0.07299007137644689
Accuracy 0.0


In [191]:
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ORCHIDS   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                 1.185e+04
Date:                Sat, 13 Apr 2024   Prob (F-statistic):               0.00
Time:                        23:51:06   Log-Likelihood:                 17028.
No. Observations:               20002   AIC:                        -3.405e+04
Df Residuals:                   20000   BIC:                        -3.404e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.001     -0.405      0.686      -0.002       0.001
x1             0.0795      0.001    108.841      0.000       0.078       0.081
==============================================================================
Omnibus:                    31561.958   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        193723212.881
Skew:                          -9.227   Prob(JB):                         0.00
Kurtosis:                     484.772   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [130]:
Data_df.head()

,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY,HUMIDITY_60_80_trans
timestamp,,,,,,,,
0,1200.00,1.5,10.5,-2.0,2500.0000,79.00000,-1,19.00000
100,1201.75,1.5,9.5,-2.0,2499.4197,79.00410,-1,19.00410
200,1201.75,1.5,9.5,-2.0,2498.8457,79.00821,-1,19.00821
300,1201.75,1.5,9.5,-2.0,2498.2780,79.01234,-1,19.01234
400,1201.75,1.5,9.5,-2.0,2497.7166,79.01649,-1,19.01649


In [127]:
X_4 = Data_df[['SUNLIGHT']].rolling(10).mean().fillna(0)
y_4 = Data_df['ORCHIDS']

In [192]:
train_test_split_idx = int(X.shape[0]*2/3)
X_train, X_test = X_4.iloc[:train_test_split_idx+1], X_4.iloc[train_test_split_idx+1:]
y_train, y_test = y_4.iloc[:train_test_split_idx+1], y_4.iloc[train_test_split_idx+1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


ols = sm.OLS(y_train, X_train)
#ols.fit(X_train, y_train)
results4 = ols.fit()

y_pred = results4.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
mean_ae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_ae)

accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.03)
print("Accuracy",accuracy_rate)


RMSE: 60.438358
Mean Absolute Error: 53.65111271442331
Accuracy 22.17778222177782


In [193]:
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ORCHIDS   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                 1.459e+04
Date:                Sat, 13 Apr 2024   Prob (F-statistic):               0.00
Time:                        23:51:54   Log-Likelihood:            -1.1014e+05
No. Observations:               20002   AIC:                         2.203e+05
Df Residuals:                   20000   BIC:                         2.203e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1134.8012      0.421   2692.655      0.000    1133.975    1135.627
x1            50.9043      0.421    120.786      0.000      50.078      51.730
==============================================================================
Omnibus:                     2165.679   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2944.091
Skew:                          -0.892   Prob(JB):                         0.00
Kurtosis:                       3.592   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [180]:
def transform_timestamp(timestamp, cycle_length=24*60*60*100):
    # Calculate angle based on position within cycle
    angle = 2 * np.pi * (timestamp % cycle_length) / cycle_length
    
    # Apply trigonometric transformation (use either sine or cosine)
    return np.sin(angle)

In [181]:
#cycle_length = 24 * 60 * 60

In [182]:
Data_df['SIN_TIMESTAMP'] = Data_df.index.map(transform_timestamp)

In [229]:
#.pct_change().fillna(0)
#, Data_df['SIN_TIMESTAMP'].diff().fillna(0)
X_5 = pd.concat([Data_df['SUNLIGHT'].pct_change().fillna(0) * 100
                                        , (Data_df['SUNLIGHT']*Data_df['SIN_TIMESTAMP']).diff().fillna(0)  * 100 ], axis=1)
X_5 = X_5.rename(columns={0: '(SUNLIGHT*SIN_TIMESTAMP)_DIFF'})
y_5 = Data_df['ORCHIDS'].pct_change().fillna(0) * 100

In [231]:
train_test_split_idx = int(X.shape[0]*2/3)
X_train, X_test = X_5.iloc[:train_test_split_idx+1], X_5.iloc[train_test_split_idx+1:]
y_train, y_test = y_5.iloc[:train_test_split_idx+1], y_5.iloc[train_test_split_idx+1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


ols = sm.OLS(y_train, X_train)
#ols.fit(X_train, y_train)
results5 = ols.fit()

y_pred = results5.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
mean_ae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_ae)

accuracy_rate = custom_accuracy(y_test, y_pred, tolerance=0.01)
print("Accuracy",accuracy_rate)


RMSE: 0.208094
Mean Absolute Error: 0.07446454761209427
Accuracy 0.0


In [233]:
pd.DataFrame(y_pred).describe()

,0
count,10001.000000
mean,-0.001199
std,0.110697
min,-11.070196
25%,-0.000560
50%,-0.000343
75%,0.000360
max,0.001119


In [234]:
results5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ORCHIDS   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     6432.
Date:                Sun, 14 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:02:24   Log-Likelihood:                 19095.
No. Observations:               20002   AIC:                        -3.818e+04
Df Residuals:                   19999   BIC:                        -3.816e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.001     -1.009      0.313      -0.002       0.001
x1            -0.0129      0.010     -1.253      0.210      -0.033       0.007
x2             0.0876      0.010      8.507      0.000       0.067       0.108
==============================================================================
Omnibus:                      191.858   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              320.859
Skew:                           0.008   Prob(JB):                     2.12e-70
Kurtosis:                       3.620   Cond. No.                         31.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""